In [13]:
import os
import sys
import librosa
import pandas as pd
import numpy as np

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ravdess = "/content/drive/MyDrive/Colab Notebooks/speech-emotion/dataset/ravdess/"
# ravdess_directory_list = os.listdir(Ravdess)
# for dir in ravdess_directory_list:
#   print(dir)

In [30]:
Ravdess = "/content/drive/MyDrive/Colab Notebooks/speech-emotion/dataset/ravdess/"
Savee = "/content/drive/MyDrive/Colab Notebooks/speech-emotion/dataset/SAVEE/AudioData/"
Feature_csv ="/content/drive/MyDrive/Colab Notebooks/speech-emotion/codes/feature-r-s.csv"

In [17]:
##create dataframe: column1 = emotion, column2 = audio path
ravdess_directory_list = os.listdir(Ravdess)

emotion_list = []
path_list = []

for dir in ravdess_directory_list:
    # Each directory represents different actor
    actor = os.listdir(Ravdess + dir)

    for audio_file in actor:
        # Take only file name and remove extention
        name_part = audio_file.split('.')[0]
        # Split the file name and take third part which represents emotion
        name_part = name_part.split('-')
        emotion_list.append(int(name_part[2]))
        # Reassemble file name with path to store for that emotion
        path_list.append(Ravdess + dir + '/' + audio_file)

# Create temporary dataframe for emotion
emotion_df = pd.DataFrame(emotion_list, columns=['Emotions'])
# Create temporary dataframe for File Path
path_df = pd.DataFrame(path_list, columns=['Path'])
# Combine above two dataframes to get final dataframe
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)


In [18]:
emotion_map = {
    1: 'neutral',
    2: 'calm',
    3: 'happy',
    4: 'sad',
    5: 'angry',
    6: 'fear',
    7: 'disgust',
    8: 'surprise'
}
Ravdess_df.Emotions.replace(emotion_map, inplace=True)
Ravdess_df.head()

,Emotions,Path
0,neutral,/content/drive/MyDrive/Colab Notebooks/speech-...
1,neutral,/content/drive/MyDrive/Colab Notebooks/speech-...
2,calm,/content/drive/MyDrive/Colab Notebooks/speech-...
3,neutral,/content/drive/MyDrive/Colab Notebooks/speech-...
4,calm,/content/drive/MyDrive/Colab Notebooks/speech-...


In [20]:
##create dataframe: column1 = emotion, column2 = audio path
emotion_list = []
path_list = []
savee_directory_list = os.listdir(Savee)

for dir in savee_directory_list:
    actor = os.listdir(Savee + dir)
    for audio_file in actor:
        emo = audio_file[:-6]
        emotion_list.append(emo)
        path_list.append(Savee + dir + '/' + audio_file)


# Create temporary dataframe for emotion
emotion_df = pd.DataFrame(emotion_list, columns=['Emotions'])
# Create temporary dataframe for File Path
path_df = pd.DataFrame(path_list, columns=['Path'])
# Combine above two dataframes to get final dataframe
Savee_df = pd.concat([emotion_df, path_df], axis=1)
Savee_df.head()

,Emotions,Path
0,a,/content/drive/MyDrive/Colab Notebooks/speech-...
1,a,/content/drive/MyDrive/Colab Notebooks/speech-...
2,a,/content/drive/MyDrive/Colab Notebooks/speech-...
3,a,/content/drive/MyDrive/Colab Notebooks/speech-...
4,f,/content/drive/MyDrive/Colab Notebooks/speech-...


In [21]:
emotion_map = {
    'n': 'neutral',
    'h': 'happy',
    'sa': 'sad',
    'a': 'angry',
    'f': 'fear',
    'd': 'disgust',
    'su': 'surprise'
}
Savee_df.Emotions.replace(emotion_map, inplace=True)
Savee_df.head()


,Emotions,Path
0,angry,/content/drive/MyDrive/Colab Notebooks/speech-...
1,angry,/content/drive/MyDrive/Colab Notebooks/speech-...
2,angry,/content/drive/MyDrive/Colab Notebooks/speech-...
3,angry,/content/drive/MyDrive/Colab Notebooks/speech-...
4,fear,/content/drive/MyDrive/Colab Notebooks/speech-...


In [22]:
data_path = pd.concat([Ravdess_df, Savee_df], axis = 0)
data_path.head()

,Emotions,Path
0,neutral,/content/drive/MyDrive/Colab Notebooks/speech-...
1,neutral,/content/drive/MyDrive/Colab Notebooks/speech-...
2,calm,/content/drive/MyDrive/Colab Notebooks/speech-...
3,neutral,/content/drive/MyDrive/Colab Notebooks/speech-...
4,calm,/content/drive/MyDrive/Colab Notebooks/speech-...


In [23]:
# Changes audio play speed according to speed_val
def change_speed(data, speed_val):
    return librosa.effects.time_stretch(data, rate=speed_val)

# Generate random noise of same size as original audio and add to the original data
def add_noise(data, noise_factor):
    noise_amp = noise_factor*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# Increase/decrease pitch by specified number of steps
def change_pitch(data, sampling_rate, pitch_factor):
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor)


In [24]:
def get_features(data, sample_rate):
    # Extract Zero Crossing Rate feature; the rate at which signal changes its sign
    feature_values = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    feature_values=np.hstack((feature_values, zcr))

    # Calculate mean of chroma short-time fourier transform; Time-frequency representation of energy distribution
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    feature_values = np.hstack((feature_values, chroma_stft))

    # Calculate mean of Mel-Frequency Cepstral Coefficients; Represents short-term power spectrum and provides timbre of sound
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    feature_values = np.hstack((feature_values, mfcc))

    # Get Root Mean Square Value of wave signal
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    feature_values = np.hstack((feature_values, rms))

    # Calculate mean of MelSpectogram; Represents audio signal's power by providing square of amplitude
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    feature_values = np.hstack((feature_values, mel))

    return feature_values


In [25]:
noise_val = 0.035
speed_val = 0.8
pitch_factor=0.7

def extract_features(path):
    # From the visualization part we saw signal starts at around 0.6 seconds and finishes within 3 seconds
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)

    # First extract features for original data and store
    feature_normal = get_features(data, sample_rate)
    feature_values = np.array(feature_normal)

    # Augment data by adding noise to the data and extract features
    feature_noise = get_features(add_noise(data, noise_val), sample_rate)
    feature_values = np.vstack((feature_values, feature_noise))

    # Augment data by reducing speed and changing pitch and finally extract feature
    feature_pitch = get_features(change_pitch(change_speed(data, speed_val), sample_rate, pitch_factor), sample_rate)
    feature_values = np.vstack((feature_values, feature_pitch))

    return feature_values

In [26]:
# List variables to store features and emotions temporarily to create dataframe
X, Y = [], []

# counter = 1

# For each audio extract features to create dataframe
for path, emotion in zip(data_path.Path, data_path.Emotions):
    # counter = counter +1
    # if counter > 3:
    #     break

    # For each audio we get 3 different feature arrays; first one for normal data,
    # second one for noise added data, and finally pitch changed data
    # For all three features emotion remains same
    feature_values = extract_features(path)

    # Add all three features as independent feature
    for feature_val in feature_values:
        X.append(feature_val)
        Y.append(emotion)

len(X), len(Y), data_path.Path.shape

(5760, 5760, (1920,))

In [27]:
# Generate final dataframe
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.321275,0.729664,0.750033,0.730624,0.735275,0.713529,0.660531,0.684966,0.733049,0.753972,...,4.310902e-06,3.291511e-06,2.148075e-06,2.279739e-06,5.116492e-06,8.190280e-06,0.000007,0.000005,4.245835e-07,neutral
1,0.313653,0.800385,0.818895,0.804210,0.803209,0.806036,0.707839,0.677258,0.725114,0.774453,...,4.860932e-05,4.813197e-05,4.904294e-05,4.882835e-05,5.054474e-05,5.394877e-05,0.000054,0.000051,4.680500e-05,neutral
2,0.188227,0.622090,0.699117,0.753257,0.721177,0.701753,0.682364,0.662857,0.686512,0.734021,...,8.577035e-07,9.576477e-07,7.733598e-07,5.233060e-07,3.593583e-07,9.267893e-07,0.000002,0.000001,7.754233e-08,neutral
3,0.323342,0.741108,0.752256,0.744445,0.732210,0.735576,0.684238,0.673684,0.699704,0.733065,...,1.982246e-05,1.854489e-05,2.479384e-05,8.005070e-05,4.002139e-05,3.728482e-05,0.000040,0.000023,1.570475e-06,neutral
4,0.245723,0.762177,0.752817,0.740239,0.760381,0.728605,0.688684,0.695779,0.740507,0.709607,...,1.984320e-05,1.854303e-05,2.479240e-05,8.001704e-05,4.002284e-05,3.730103e-05,0.000040,0.000023,1.578377e-06,neutral


In [31]:
Features.to_csv(Feature_csv, index=False)